In [1]:
!pip install gensim
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.4 MB/s 
     |████████████████████████████████| 596 kB 35.2 MB/s 
     |████████████████████████████████| 77 kB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 29.1 MB/s 
     |████████████████████████████████| 895 kB 37.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!ls

drive  sample_data


In [3]:
import os

os.chdir('drive/MyDrive/William_2022')

In [4]:
import gensim
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np

import logging
logging.basicConfig(level=logging.INFO) 

import matplotlib.pyplot as plt
% matplotlib inline

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm

import random
from sklearn import metrics, model_selection, preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [95]:
def seed_everything(seed=73):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # some cudnn methods can be random even after fixing the seed unless you tell it to be deterministic
    torch.backends.cudnn.deterministic = True

seed_everything(1234)

## Preprocessing


In [ ]:
data_dir = 'DATASETS/CSV/transcript_emotion_6class'

first = 1
second = 2

transcript = pd.read_csv(data_dir + f'/p{first}.p{second}a.csv')
transcript.rename(columns = {'results/alternatives/0/transcript' : 'full_transcript'}, inplace = True) 
transcript.rename(columns = {'results/alternatives/0/words/0/word' : 'word'}, inplace = True) 

only_fulltranscript = transcript[transcript['new_words'].notna()]
only_fulltranscript = only_fulltranscript.reset_index()
only_fulltranscript = only_fulltranscript.iloc[:, 28:]

In [ ]:
word_list0 = []
word_list1 = []

# for j in range(0, len(df)):
for i in range(0, len(transcript)-1):
    if (transcript.seconds[i] == transcript.seconds[i+1]):
        #print(df.iloc[:i+2, 5])
        #print(df.word[i+1])
        word_list = transcript.word[i]
        #print(word_list)
        word_list0.append(word_list)
        
        
        #print(word_list0)
        
        
        
    else:
        word_list0.append(transcript.word[i])
        word_list1.append(word_list0)
        
        word_list0=[]

word_list1.append(word_list0)
transcript['new_words'] = ''

for i in range(0, len(word_list1)):
  #print(i)
  word_list1[i] = ' '.join(word_list1[i])
  transcript.new_words[i] = word_list1[i]

transcript.to_csv(f'p{first}.p{second}.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
transcript = pd.read_csv(f'p{first}.p{second}.csv')
transcript[f'p{first}_trans'] = ''
transcript['seconds_trans'] = ''
transcript['arousal_trans'] = ''
transcript['valence_trans'] = ''
transcript['happy_trans'] = ''
transcript['angry_trans'] = ''
transcript['nervous_trans'] = ''
transcript['sad_trans'] = ''

transcript[f'p{second}_trans'] = ''
transcript['seconds2_trans'] = ''
transcript['arousal2_trans'] = ''
transcript['valence2_trans'] = ''
transcript['happy2_trans'] = ''
transcript['angry2_trans'] = ''
transcript['nervous2_trans'] = ''
transcript['sad2_trans'] = ''
fir_p = pd.read_csv(f'DATASETS/CSV/averaged_emotions/p{first}.csv')
sec_p = pd.read_csv(f'DATASETS/CSV/averaged_emotions/p{second}.csv')

for i in range(0,len(word_list1)):
  
  transcript.iloc[i, 28:35] = fir_p.iloc[i]

  
for j in range(0, len(word_list1)):  
  transcript.iloc[j, 36:43] = sec_p.iloc[j]


transcript.to_csv(f'p{first}.p{second}a.csv')

In [ ]:
data_dir = 'DATASETS/CSV/transcript_emotion_6class'

first = 1
second = 2


transcript = pd.read_csv(data_dir + f'/p{first}.p{second}a.csv')
transcript.rename(columns = {'results/alternatives/0/transcript' : 'full_transcript'}, inplace = True) 
transcript.rename(columns = {'results/alternatives/0/words/0/word' : 'word'}, inplace = True) 

only_fulltranscript = transcript[transcript['new_words'].notna()]
only_fulltranscript = only_fulltranscript.reset_index()
only_fulltranscript = only_fulltranscript.iloc[:, 28:]


In [ ]:
only_fulltranscript

,index,Unnamed: 0,Unnamed: 0.1,results/alternatives/0/confidence,full_transcript,results/alternatives/0/words/0/confidence,reult/alternative/0/word/0/endTime,startTime,word,results/channelTag,...,nervous_trans,sad_trans,p32_trans,seconds2_trans,arousal2_trans,valence2_trans,happy2_trans,angry2_trans,nervous2_trans,sad2_trans
0,0,0,0,0.884961,okay the first are maybe you cannot understand...,0.912839,3.1,2.5,okay,31,...,1.333,1.0,NaN,5.0,3.667,3.333,2.000,1.000,1.667,1.0
1,1,1,1,0.884961,NaN,0.912839,4.4,4.3,the,31,...,1.333,1.0,NaN,10.0,3.667,3.333,2.000,1.000,1.667,1.0
2,2,2,2,0.884961,NaN,0.912839,4.8,4.4,first,31,...,1.333,1.0,NaN,15.0,4.000,4.000,2.000,1.000,1.333,1.0
3,3,3,3,0.884961,NaN,0.912839,5.1,4.8,are,31,...,1.333,1.0,NaN,20.0,4.000,4.000,2.000,1.000,1.333,1.0
4,4,4,4,0.884961,NaN,0.912839,6.8,6.5,maybe,31,...,1.333,1.0,NaN,25.0,3.333,3.000,1.667,1.333,1.667,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,111,111,111,0.863448,NaN,0.912839,74.1,74.0,in,31,...,1.000,1.0,NaN,560.0,3.333,3.000,1.333,1.333,1.333,1.0
112,112,112,112,0.863448,NaN,0.644002,74.8,74.1,myself,31,...,1.000,1.0,NaN,565.0,3.667,3.000,1.333,1.333,1.333,1.0
113,113,113,113,0.880028,yeah and finally from a legal point of view K...,0.912839,77.0,76.6,yeah,31,...,1.000,1.0,NaN,570.0,3.667,3.000,1.333,1.333,2.000,1.0
114,114,114,114,0.880028,NaN,0.912839,78.3,77.8,and,31,...,1.000,1.0,NaN,575.0,3.333,3.000,1.333,1.333,2.000,1.0


In [ ]:
full_transcript = pd.DataFrame()

In [ ]:
data_dir = 'DATASETS/CSV/transcript_emotion_6class/'

first = 1
second = 2



for csv in range(0, len(os.listdir(data_dir))):
  #print(csv)
  transcript = pd.read_csv(f'{data_dir}/{os.listdir(data_dir)[csv]}')
  #print(csv)
  #transcript = pd.read_csv(data_dir + f'/p{first}.p{second}a.csv')
  #print(transcript)
  only_fulltranscript = transcript[transcript['new_words'].notna()]
  only_fulltranscript = only_fulltranscript.reset_index()
  only_fulltranscript_fir = only_fulltranscript[['new_words',	'seconds_trans',	'arousal_trans',	'valence_trans',	'happy_trans',	'angry_trans',	'nervous_trans',	'sad_trans']]
  only_fulltranscript_sec = only_fulltranscript[['seconds2_trans',	'arousal2_trans',	'valence2_trans',	'happy2_trans',	'angry2_trans',	'nervous2_trans',	'sad2_trans']]
  only_fulltranscript_concat = pd.concat([only_fulltranscript_fir, only_fulltranscript_sec], axis = 1)
  #print(only_fulltranscript_fir)
  #print(only_fulltranscript_sec)
  full_transcript = full_transcript.append(only_fulltranscript_concat)
  #print(full_transcript2)
  
  #print(full_transcript2)
  first +=1
  second +=1



  
full_transcript.to_csv('full_transcript.csv',sep = ',', index = False)




## Word Embeddings

In [74]:
full_transcript = pd.read_csv('DATASETS/CSV/full_transcript_concat.csv')

In [75]:
full_transcript['arousal_trans'] = full_transcript['arousal_trans'].round()
full_transcript['arousal2_trans'] = full_transcript['arousal2_trans'].round()

full_transcript['valence_trans'] = full_transcript['valence_trans'].round()
full_transcript['valence2_trans'] = full_transcript['valence2_trans'].round()

full_transcript['happy_trans'] = full_transcript['happy_trans'].round()
full_transcript['happy2_trans'] = full_transcript['happy2_trans'].round()

full_transcript['angry_trans'] = full_transcript['angry_trans'].round()
full_transcript['angry2_trans'] = full_transcript['angry2_trans'].round()

full_transcript['nervous_trans'] = full_transcript['nervous_trans'].round()
full_transcript['nervous2_trans'] = full_transcript['nervous2_trans'].round()

full_transcript['sad_trans'] = full_transcript['sad_trans'].round()
full_transcript['sad2_trans'] = full_transcript['sad2_trans'].round()


In [76]:
full_transcript

,new_words,seconds_trans,arousal_trans,valence_trans,happy_trans,angry_trans,nervous_trans,sad_trans,seconds2_trans,arousal2_trans,valence2_trans,happy2_trans,angry2_trans,nervous2_trans,sad2_trans
0,all right first one the state for this accepti...,5.0,1.0,3.0,2.0,1.0,1.0,1.0,5.0,3.0,3.0,1.0,1.0,2.0,1.0
1,arrived in this especially in many Arab Jesus ...,10.0,2.0,3.0,2.0,1.0,1.0,1.0,10.0,3.0,3.0,1.0,1.0,2.0,1.0
2,Jeju Allah first of all I want to mention that...,15.0,2.0,3.0,1.0,1.0,1.0,1.0,15.0,3.0,3.0,1.0,1.0,2.0,1.0
3,movement in South Korea against against this y...,20.0,2.0,3.0,1.0,1.0,1.0,1.0,20.0,4.0,3.0,1.0,1.0,2.0,1.0
4,pretty much standard phobic activity because,25.0,2.0,3.0,1.0,1.0,1.0,1.0,25.0,3.0,3.0,1.0,1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1858,The Courier what curse income,560.0,3.0,4.0,1.0,1.0,1.0,1.0,560.0,3.0,3.0,1.0,1.0,1.0,1.0
1859,cannot there with this issue and cannot,565.0,3.0,4.0,1.0,1.0,1.0,1.0,565.0,4.0,3.0,1.0,1.0,1.0,1.0
1860,change the no in this issue yeah,570.0,3.0,4.0,1.0,1.0,1.0,1.0,570.0,4.0,3.0,1.0,1.0,2.0,1.0
1861,your suggestion is very proper but there's no ...,575.0,3.0,4.0,1.0,1.0,1.0,1.0,575.0,3.0,3.0,1.0,1.0,2.0,1.0


In [77]:
print(full_transcript['arousal_trans'].value_counts())
print(full_transcript['valence_trans'].value_counts())
print(full_transcript['happy_trans'].value_counts())
print(full_transcript['angry_trans'].value_counts())
print(full_transcript['nervous_trans'].value_counts())
print(full_transcript['sad_trans'].value_counts())

3.0    1201
2.0     406
4.0     255
1.0       1
Name: arousal_trans, dtype: int64
3.0    1452
4.0     239
2.0     172
Name: valence_trans, dtype: int64
1.0    1231
2.0     608
3.0      24
Name: happy_trans, dtype: int64
1.0    1491
2.0     370
3.0       2
Name: angry_trans, dtype: int64
1.0    907
2.0    890
3.0     66
Name: nervous_trans, dtype: int64
1.0    1803
2.0      60
Name: sad_trans, dtype: int64


In [78]:
print(full_transcript['arousal_trans'].unique())
print(full_transcript['arousal2_trans'].unique())

print(full_transcript['valence_trans'].unique())
print(full_transcript['valence2_trans'].unique())

print(full_transcript['happy_trans'].unique())
print(full_transcript['happy2_trans'].unique())

print(full_transcript['angry_trans'].unique())
print(full_transcript['angry2_trans'].unique())

print(full_transcript['nervous_trans'].unique())
print(full_transcript['nervous2_trans'].unique())

print(full_transcript['sad_trans'].unique())
print(full_transcript['sad2_trans'].unique())

[1. 2. 3. 4.]
[3. 4. 2.]
[3. 2. 4.]
[3. 4. 2. 5.]
[2. 1. 3.]
[1. 2. 3.]
[1. 2. 3.]
[1. 2.]
[1. 2. 3.]
[2. 1. 3.]
[1. 2.]
[1. 2.]


In [79]:
print(full_transcript['arousal2_trans'].value_counts())
print(full_transcript['valence2_trans'].value_counts())
print(full_transcript['happy2_trans'].value_counts())
print(full_transcript['angry2_trans'].value_counts())
print(full_transcript['nervous2_trans'].value_counts())
print(full_transcript['sad2_trans'].value_counts())

3.0    1327
2.0     334
4.0     202
Name: arousal2_trans, dtype: int64
3.0    1574
4.0     152
2.0     131
5.0       6
Name: valence2_trans, dtype: int64
1.0    1138
2.0     682
3.0      43
Name: happy2_trans, dtype: int64
1.0    1716
2.0     147
Name: angry2_trans, dtype: int64
1.0    950
2.0    826
3.0     87
Name: nervous2_trans, dtype: int64
1.0    1792
2.0      71
Name: sad2_trans, dtype: int64


In [21]:
emotion_labels = {
             'happy' : [1., 1.333, 1.667, 2., 2.333, 2.667, 3., 3.333],
             'angry' : [1., 1.333, 1.667, 2., 2.333, 2.667],
             'nervous'  : [1., 1.333, 1.667, 2., 2.333, 2.667, 3., 3.333],
             'sad' : [1., 1.333, 1.667, 2., 2.333] }

In [ ]:
# --- rounded
emotion_labels_round = {
    'happy'   : [1.0, 2.0, 3.0],
    'angry'   : [1.0, 2.0, 3.0],
    'nervous' : [1.0, 2.0, 3.0],
    'sad'     : [1.0, 2.0]
}

In [ ]:
emotion_labels_a_v = {
    'arousal'   : [1.333, 1.667, 2., 2.333, 2.667, 3., 3.333, 3.667, 4., 4.333],
    'valence'   : [2., 2.333, 2.667, 3., 3.333, 3.667, 4., 4.333, 4.667]
}

In [ ]:
# --- rounded
emotion_labels_a_v_round = {
    'arousal'   : [1.0, 2.0, 3.0, 4.0],
    'valence'   : [2.0, 3.0, 4.0, 5.0]
}

In [32]:
emotion_labels_ratings = {
    1.    : 'very low',
    1.333 : '',
    1.667 : '',
    2.    : 'low',
    2.333 : '',
    2.667 : '',
    3.    : 'high',
    3.333 : '',
    }

In [ ]:
# --- rounded
emotion_labels_ratings_rounded = {
    1.0   : 'very low',
    2.0   : 'low',
    3.0   : 'high'
}

In [33]:
emotion_labels_a_v_ratings = {
    1.333 : 'very low',
    1.667 : '',
    2.    : 'low',
    2.333 : '',
    2.667 : '',
    3.    : 'neutral',
    3.333 : '',
    3.667 : '',
    4.    : 'high',
    4.333 : '',
    4.667 : 'very high'
}

In [ ]:
# --- rounded
emotions_label_a_v_ratings_rounded = {
    1.0 : 'very low',
    2.0 : 'low',
    3.0 : 'neutral',
    4.0 : 'high',
    5.0 : 'very high'
}

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

full_transcript.happy_trans = le.fit_transform(full_transcript.happy_trans)

full_transcript.head

In [35]:
full_transcript.shape

(1863, 15)

## fir_p happy - sad

In [100]:
# --- emotions happy - sad fir_p
from sklearn.model_selection import train_test_split

y = full_transcript.drop(['seconds_trans','new_words', 'seconds2_trans', 'arousal_trans', 'arousal2_trans', 'valence_trans', 'valence2_trans', 'happy2_trans', 'angry2_trans', 'nervous2_trans', 'sad2_trans'], 1)
X = full_transcript[['new_words']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [87]:
y

,new_words,happy_trans,angry_trans,nervous_trans,sad_trans
0,all right first one the state for this accepti...,2.0,1.0,1.0,1.0
1,arrived in this especially in many Arab Jesus ...,2.0,1.0,1.0,1.0
2,Jeju Allah first of all I want to mention that...,1.0,1.0,1.0,1.0
3,movement in South Korea against against this y...,1.0,1.0,1.0,1.0
4,pretty much standard phobic activity because,1.0,1.0,1.0,1.0
...,...,...,...,...,...
1858,The Courier what curse income,1.0,1.0,1.0,1.0
1859,cannot there with this issue and cannot,1.0,1.0,1.0,1.0
1860,change the no in this issue yeah,1.0,1.0,1.0,1.0
1861,your suggestion is very proper but there's no ...,1.0,1.0,1.0,1.0


In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [110]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1490, 1) (373, 1) (1490, 4) (373, 4)


In [111]:
# --- rounded
emotions_label_a_v_ratings_rounded = {
    1.0 : 'very low',
    2.0 : 'low',
    3.0 : 'neutral',
    4.0 : 'high',
    5.0 : 'very high'
}

In [112]:
n_labels = len(emotions_label_a_v_ratings_rounded)

In [115]:
y_train.iloc[0][['happy_trans', 'sad_trans']]

happy_trans    2
sad_trans      1
Name: 873, dtype: int64

In [114]:
y_train = y_train.astype(int)

In [107]:
def one_hot_encoder(df):
    one_hot_encoding = []
    for i in tqdm(range(len(df))):
        temp = [0]*n_labels
        label_indices = df.iloc[i]['happy_trans']
        for index in label_indices:
            temp[index] = 1
        one_hot_encoding.append(temp)
    return pd.DataFrame(one_hot_encoding)

In [116]:
train_ohe_labels = one_hot_encoder(y_train)
valid_ohe_labels = one_hot_encoder(y_test)

  0%|          | 0/1490 [00:00<?, ?it/s]

TypeError: ignored

In [56]:
x_train = tokenizer(
    text=X_train.new_words.tolist(),
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    padding=True, 
    return_tensors='pt',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

In [57]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased', 
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

## sec_p happy - sad

In [46]:
# --- emotions happy - sad sec_p

y = full_transcript.drop(['new_words', 'seconds_trans', 'seconds2_trans', 'arousal2_trans', 'valence2_trans', 'valence_trans', 'arousal_trans', 'happy_trans', 'angry_trans', 'nervous_trans', 'sad_trans'], 1)
X = full_transcript[['new_words']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [47]:
y

,happy2_trans,angry2_trans,nervous2_trans,sad2_trans
0,1.333,1.000,1.667,1.0
1,1.333,1.000,2.000,1.0
2,1.333,1.000,2.333,1.0
3,1.333,1.000,2.333,1.0
4,1.333,1.000,2.333,1.0
...,...,...,...,...
1858,1.333,1.333,1.333,1.0
1859,1.333,1.333,1.333,1.0
1860,1.333,1.333,2.000,1.0
1861,1.333,1.333,2.000,1.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [ ]:
x_train = tokenizer(
    text=X_train.new_words.tolist,
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    padding=True, 
    return_tensors='pt',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

## fir_p Arousal and Valence

In [48]:
# --- arousal and valence fir_p

from sklearn.model_selection import train_test_split

y = full_transcript.drop(['new_words', 'seconds_trans', 'seconds2_trans', 'happy_trans', 'happy2_trans', 'angry_trans', 'angry2_trans', 'nervous_trans', 'nervous2_trans', 'sad_trans', 'sad2_trans', 'arousal2_trans', 'valence2_trans'], 1)
X = full_transcript[['new_words']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


In [49]:
y

,arousal_trans,valence_trans
0,1.333,2.667
1,1.667,2.667
2,2.000,2.667
3,2.000,3.000
4,2.000,3.000
...,...,...
1858,3.333,4.000
1859,3.333,4.000
1860,3.333,4.000
1861,3.333,4.000


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [ ]:
x_train = tokenizer(
    text=X_train.new_words.tolist,
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    padding=True, 
    return_tensors='pt',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

## sec_p Arousal and Valence

In [50]:
# --- arousal and valence sec_p

from sklearn.model_selection import train_test_split

y = full_transcript.drop(['new_words', 'seconds_trans', 'seconds2_trans', 'happy_trans', 'happy2_trans', 'angry_trans', 'angry2_trans', 'nervous_trans', 'nervous2_trans', 'sad_trans', 'sad2_trans', 'arousal_trans', 'valence_trans'], 1)
X = full_transcript[['new_words']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


In [51]:
y

,arousal2_trans,valence2_trans
0,3.000,2.667
1,3.333,3.000
2,3.333,2.667
3,3.667,2.667
4,3.333,2.667
...,...,...
1858,3.333,3.000
1859,3.667,3.000
1860,3.667,3.000
1861,3.333,3.000


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [12]:
X_test

,new_words
615,is giving me a minute to my Legion for nine da...
558,sure with enough intent the government would b...
136,should be guaranteed those humanitarian issues...
1563,very young and strong male refugees and
1612,Yum Yum music cat has more possibility to
...,...
293,is these people are looking for cheaper or the...
658,yeah basically what you're trying to say well ...
918,who's not Christian they still don't know it's...
777,isn't all work isn't ready to accept these ref...


In [26]:
x_train = tokenizer(
    text=X_train.new_words.tolist,
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    padding=True, 
    return_tensors='pt',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

In [30]:
x_train['input_ids'].shape

torch.Size([1490, 55])

In [31]:
x_train['attention_mask']

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [ ]:
tokenized_text_list = []

indexed_tokens_list = []

segment_ids_list = []

tokens_tensor_list = []

segment_tensors_list = []

# def preprocess_text(only_fulltranscript):
for i in range(0, len(full_transcript)):
  text = full_transcript.new_words[i]
  marked = '[CLS]' + text + '[SEP]'

  tokenized_text = tokenizer.tokenize(marked)
  tokenized_text_list.append(tokenized_text)
  #print(tokenized_text)

  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  indexed_tokens_list.append(indexed_tokens)

  # for tup in zip(tokenized_text, indexed_tokens):
  #   print('{:<12} {:>6,}'.format(tup[0], tup[1]))

  segment_ids = [i] * len(tokenized_text)
  segment_ids_list.append(segment_ids)
  #print(segment_ids)
  
  tokens_tensor = torch.tensor([indexed_tokens])
  tokens_tensor_list.append(tokens_tensor)

  segment_tensors = torch.tensor([segment_ids])
  segment_tensors_list.append(segment_tensors)

  # return tokenized_text, indexed_tokens, segment_ids, tokens_tensor, segment_tensors


In [ ]:
segment_tensors_list

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [ ]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased', 
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          